In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../") # go to parent dir

import numpy as np
import torch
from torch import nn, optim
import matplotlib.pyplot as plt

from itertools import product

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Class balance estimation from weak supervision

In this notebook, we'll use a special class, `ClassBalanceModel`, for estimating the _class balance_, i.e. $P(Y)$, from unlabeled data and a set of _conditionally independent_ labeling functions.  We rely on this conditional independence assumption in this approach (note that these LFs could be a subset of the larger set of LFs used in the `LabelModel`, selected to satisfy this assumption).

**Note that this is an advanced tutorial, and thus will go into slightly more detail in certain aspects, and skip over some basics in other parts.**

Note also that:
* The LFs in this tutorial do not abstain, however the `ClassBalanceModel` does handle abstaining LFs.
* We directly generate a three-way overlaps matrix $O$ as a toy synthetic example; however, the `ClassBalanceModel` can also take in an empirical label matrix $L$ and form $O$ internally.

For further details, see the tests in `tests/metal/label_model/test_class_balance.py`.

In [2]:
np.random.seed(123)
torch.manual_seed(123)

### Step 1: Generate the true class balance to be recovered

In [3]:
K = 3

# Generate the true class balance to be recovered
p_Y = np.random.random(K)
p_Y /= p_Y.sum()
p_Y

array([0.57585137, 0.23658438, 0.18756425])

### Step 2: Generate the true conditional probability tables (CPTs) for the LFs

We start by generating the conditional probability table $C$ for $M$ labeling functions (LFs) which we assume are conditionally independent given the latent true label $Y$, i.e. $\lambda_i \perp \lambda_{j \neq i} | Y$.
We have:
$$
C_{i,y',y} = P(\lambda_i = y' | Y = y)
$$

Note that this table should be normalized such that:
$$
\sum_{y'} C_{i,y',y} = 1
$$

Note also that we skew the diagonal entries to roughly enforce our other assumption here: that the LFs are (on average) better than random. We will later rely on this assumption for breaking a final column-wise symmetry in our class balance estimation problem.

In [4]:
M = 25
Cs = []
for i in range(M):
    a = np.random.random((K, K)) + (K-1)*np.eye(K)
    Cs.append( a @ np.diag(1 / a.sum(axis=0)) )
C = np.array(Cs)
assert np.all(np.abs(C.sum(axis=1) - 1) < 1e-5)
C[0]

array([[0.6501496 , 0.19198758, 0.11645918],
       [0.24992739, 0.71643667, 0.13237551],
       [0.09992301, 0.09157576, 0.7511653 ]])

### Step 3: Generate the _three-way_ overlaps tensor $O$ of conditionally-independent LFs

Now we can directly generate $O$.
By our conditional independence assumption, for distinct $i,j,k$, we have:
$$
P(\lambda_i = y', \lambda_j = y'', \lambda_k = y''' | Y = y) = C_{i,y',y} C_{j,y'',y} C_{j,y''',y}
$$

Thus we have:
$$
O_{i,j,k,y',y'',y'''} = \sum_y P(Y=y) C_{i,y',y} C_{j,y'',y} C_{j,y''',y}
$$

Note that we explicitly mask out entries for which $i,j,k$ are not distinct; however, note that this is redundant given that this masking also occurs in the `ClassBalanceModel` class we will use!

In [5]:
# Compute mask
mask = torch.ones((M, M, M, K, K, K)).byte()
for i, j, k in product(range(M), repeat=3):
    if len(set((i, j, k))) < 3:
        mask[i, j, k, :, :, :] = 0

Note that we use `einsum`, which is great!

In [6]:
%%time
O = np.einsum('aby,cdy,efy,y->acebdf', C, C, C, p_Y)
O = torch.from_numpy(O).float()
O[1-mask] = 0

CPU times: user 75.7 ms, sys: 4.35 ms, total: 80.1 ms
Wall time: 22.6 ms


### Step 4: Recover the class balance $P(Y)$ using `ClassBalanceModel`

In [7]:
%%time
from metal.label_model.class_balance import ClassBalanceModel

model = ClassBalanceModel(K, abstains=False)
model.train_model(O=O, verbose=False)

print(f"Estimated class balance: {model.class_balance}")
print(f"True class balance: {p_Y}")
print(f"Avg. error: {np.mean(np.abs(model.class_balance - p_Y))}")

Estimated class balance: [0.5758515  0.23658441 0.18756407]
True class balance: [0.57585137 0.23658438 0.18756425]
Avg. error: 1.1204306531638804e-07
CPU times: user 32.4 s, sys: 857 ms, total: 33.2 s
Wall time: 5.55 s


Again, for further details and examples, see:
* The source code, in `metal/label_model/class_balance.py`
* The tests, in `tests/metal/label_model/test_class_balance.py`